<a href="https://colab.research.google.com/github/CodingTomo/PyTorch-Tutorials/blob/master/PyTorch_Convolutional_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Convolutional Neural Networks

Le reti neurali convoluzionali (CNN) sono un particolare tipo di reti neurali profonde la cui caratteristica è quella di operare sui dati principalmente tramite l'operazione di **convoluzione**.

La convoluzione è un operazione estramemente potente usata in vari settori della matematica, della fisica e dell'ingegneria. 

[Ulteriori informazioni sulla convoluzione](https://en.wikipedia.org/wiki/Convolution)

Parlare in maniera diffusa della definizione e delle varie proprietà della convoluzione richiederebbe molto tempo, quindi cerchiamo solo di capire cosa avviene operativamente all'interno di una CNN che lavora con delle di immagini.

Supponiamo di avere:
*   una matrice di input $A$ di dimensione 5x5;
*   una matrice di filtro $B$ di dimensione 3x3.

La convoluzione fra $A$ e $B$ significa calcolare una sorta di prodotto scalare di A con B mentre B "scorre" su A. La seguente immagine è molto chiara.

<p>
<img src="Immagini/conv.jpg" width="250" style="margin-left: auto;margin-right: auto;display: block;" />
</p>



In [0]:
import sys
from collections import OrderedDict
from matplotlib import pyplot as plt 

import torch
import torch.nn as nn
import torch.optim as optim

torch.set_printoptions(precision=3)

Per eseguire una convoluzione tra matrici sfuttiamo la funzione
`
torch.nn.functional.conv2d
`.

Osserviamo che la funzione richiede che il tensore di input e il filtro o *kernel* debbano avere dimensione $4$. In particolare, tali dimensioni rappresentano

*   **Input**: Batch size, Num channels, Input height, Input width;
*   **Filtro**: Num output channels, Num input channels, Filter height, Filter width.





In [0]:
input = torch.Tensor([[3,3,2,1,0], [0,0,1,3,1], [3,1,2,2,3], [2,0,0,2,2], [2,0,0,0,1]]).view(1,1,5,5)
kernel = torch.Tensor([[0,1,2],[2,2,0],[0,1,2]]).view(1,1,3,3)

conv_result = torch.nn.functional.conv2d(input, kernel)
print("Il risultato della convoluzione del kernel sull'input è: \n {}".format(conv_result.squeeze().numpy()))

L'operazione di convoluzione è spesso seguita da un operazione di **pooling**. Questa operazione ha lo scopo di sintetizzare le feature rilevate nell'input a seguito della funzione di attivazione susseguente allo strato convolutivo.

IMMAGINE

*In all cases, pooling helps to make the representation become approximately invariant to small translations of the input. Invariance to translation means that if we translate the input by a small amount, the values of most of the pooled outputs do not change.* 

[DeepLearningBook](http://www.deeplearningbook.org)

Nell'esempio che segue operiamo in *MaxPool* 2x2 su `conv_result`. In pratica con un scorriamo una matrice senza pesi sull'input e selezioniamo sempre il massimo fra i $4$ numeri sezionati.

In [0]:
pool_result = torch.nn.functional.max_pool2d(conv_result, kernel_size=(2,2), stride=1)
print("Il risultato della convoluzione del kernel sull'input è: \n {}".format(pool_result.squeeze().numpy()))

Negli esempi successivi vediamo come particolari **filtri** operano per convoluzione su **immagini** reali.

In [0]:
from PIL import Image

image = Image.open("Immagini/staccionata.jpg").convert('L')
print(image.size)
image

**Osservazione**: la funzione `convert('L')` trasforma l'immagine in bianco e nero. La trasformazione è una somma pesata dei canali RGB di input.



Definiamo di seguito tre **filtri** e operiamo la convoluzione con la nostra immagine di input. I filtri hanno uno **scopo** ben preciso: sfuocare l'immagine, evidenziare linee orizzontali e evidenziare linee verticali.

In [0]:
from torchvision.transforms.functional import to_tensor, to_pil_image

image_tensor = to_tensor(image)
input = image_tensor.unsqueeze(0)
kernel_out_focus = torch.Tensor([[[
                         [1/25,1/25,1/25,1/25,1/25],
                         [1/25,1/25,1/25,1/25,1/25],
                         [1/25,1/25,1/25,1/25,1/25],
                         [1/25,1/25,1/25,1/25,1/25],
                         [1/25,1/25,1/25,1/25,1/25]
                         ]]])

kernel_hor=torch.Tensor([[[
                         [-1,-1,-1],
                         [0,0,0],
                         [1,1,1]
                         ]]])

kernel_vert=torch.Tensor([[[
                         [-1,0,1],
                         [-1,0,1],
                         [-1,0,1]
                         ]]])

In [0]:
out = torch.nn.functional.conv2d(input, kernel_out_focus)
norm_out = (out - out.min()) / (out.max() - out.min())
to_pil_image(norm_out.squeeze())

In [0]:
out = torch.nn.functional.conv2d(input, kernel_hor)
norm_out = (out - out.min()) / (out.max() - out.min())
to_pil_image(norm_out.squeeze())

In [0]:
out = torch.nn.functional.conv2d(input, kernel_vert)
norm_out = (out - out.min()) / (out.max() - out.min())
to_pil_image(norm_out.squeeze())

### Esempio di classificazione usando il dataset MNIST

Come nel precedente tutorial, riproviamo a classificare le immagini delle cifre manoscritte del dataset MNIST usando questa volta una rete convoluzionale. L'architettura della rete che useremo è una verisione modificata della famosa [LeNet-5](https://en.wikipedia.org/wiki/Convolutional_neural_network).



**Architettura**

+ Strati convoluzionali:


| Layer       | Name | Input channels | Output channels | Kernel | stride |
| ----------- | :--: | :------------: | :-------------: | :----: | :----: |
| Convolution |  conv_0  |       1        |        6        |  5x5   |   1    |
| ReLU        |      |       6        |        6        |        |        |
| MaxPooling  |  pool_0  |       6        |        6        |  2x2   |   2    |
| Convolution |  conv_1  |       6        |       16        |  5x5   |   1    |
| ReLU        |      |       16       |       16        |        |        |
| MaxPooling  |  pool_1  |       16       |       16        |  2x2   |   2    |
| Convolution |  conv_2  |       6        |       120       |  5x5   |   1    |
| ReLU        |      |      120       |       120       |        |        |


+ Strati *fully connected*:

| Layer      | Name | Input size | Output size |
| ---------- | :--: | :--------: | :---------: |
| Linear     |  lin_0  |    120     |     84      |
| ReLU       |      |            |             |
| Linear     |  lin_1  |     84     |     10      |
| LogSoftmax |      |            |             |


In [0]:
import torch
import torch.nn.functional as funct
import numpy as np
import matplotlib.pyplot as plt

Iniziamo definendo la **rete neurale** seguendo la tabella precedente. Osserviamo che il parametro *stride* misura di quanti pixel deve scorrere il filtro durante le operazioni di convoluzione e di pooling; il valore di default è $1$.

In [0]:
class LeNet5(nn.Module):
  def __init__(self):
        super(LeNet5,self).__init__()

        self.conv_0 = nn.Conv2d(1, 6, kernel_size=(5, 5),stride=1)
        self.pool_0 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.conv_1 = nn.Conv2d(6,16,kernel_size=(5,5),stride=1)
        self.pool_1 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.conv_2 = nn.Conv2d(16,120, kernel_size=(5,5), stride=1)
        self.lin_0 = nn.Linear(120,84)
        self.lin_1 = nn.Linear(84,10)

  def forward(self,image):
    out = funct.relu(self.conv_0(image))
    out = funct.relu(self.conv_1(self.pool_0(out)))
    out = funct.relu(self.conv_2(self.pool_1(out)))
    out = out.view(image.shape[0], -1)
    out = funct.relu(self.lin_0(out))
    out = funct.log_softmax(self.lin_1(out),dim=-1)
    return out

Riassumiamo le caratteristiche della nostra rete neurale.

In [0]:
MyLeNet = LeNet5()
print(MyLeNet)

In [0]:
named_params = list(MyLeNet.named_parameters())
print('Le entità addrestabili della rete sono:')
for name, param in named_params:
    print("  %s:\t%s" % (name, param.shape))
print ('\n Il numero totale di parametri addestrabili è', sum(p.numel() for p in MyLeNet.parameters() if p.requires_grad))

Testiamo la rete con un input casuale, sfruttando l'inizializzazione di default dei vari parametri addestrabili.

In [0]:
input = torch.randn(1, 1, 32, 32)  # batch_size, num_channels, height, width
out = MyLeNet(input)
print("Log-Probabilities: \n%s\n" % out)
print("Probabilities: \n%s\n" % torch.exp(out))
print("out.shape: \n%s" % (out.shape,))

Definiamo la **funzione di training**.

In [0]:
def train_cnn(model, train_loader, test_loader, device, num_epochs, lr=0.1):

    train_acc_list = np.array([])
    test_acc_list = np.array([])
    time_list = np.array([])


    # Ottimizzatore e funzione di errore
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        print("=" * 40, "Starting epoch %d" % (epoch + 1), "=" * 40)
        
        model.train() # impostiamo la modalità addestramento al modulo.
                
        # cicli concatenati: per ogni batch andiamo ad aggiustare i pesi
        for batch_idx, (data, labels) in enumerate(train_loader):
            data, labels = data.to(device), labels.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            #stampiamo l'andamento ogni 75 batch 
            if batch_idx % 75 == 0:
                print("Batch %d/%d, Loss=%.4f" % (batch_idx, len(train_loader), loss.item()))
        
        # Compute the train and test accuracy at the end of each epoch
        train_acc = accuracy(model, train_loader, device)
        test_acc = accuracy(model, test_loader, device)

        train_acc_list = np.append(train_acc_list, [train_acc])
        test_acc_list = np.append(test_acc_list, [test_acc])
        time_list = np.append(time_list, [epoch+1])

    return train_acc_list, test_acc_list, time_list

Definiamo una **metrica** di valutazione delle performance.

In [0]:
def accuracy(model, dataloader, device):
  
    model.eval() # impostiamo la modalità test al modulo.
    
    num_correct = 0
    num_samples = 0
    with torch.no_grad():  
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device)

            predictions = model(data).max(1)[1]  
            num_correct += (predictions == labels).sum().item()
            num_samples += predictions.shape[0]
        
    return num_correct / num_samples

Importiamo i **dati** e costruiamo i **batch**.

In [0]:
from torchvision import datasets, transforms

transformations = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_data = datasets.MNIST('./data', 
                            train = True, 
                            download = True,
                            transform = transformations)

test_data = datasets.MNIST('./data', 
                            train = False, 
                            download = True,
                            transform = transformations)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1024, shuffle=False)

Visualizziamo qualche esempio di addestamento.

In [0]:
plt.figure(figsize=(16,9))
data, target = next(iter(train_loader))
for i in range(10):
    img = data.squeeze(1)[i]
    plt.subplot(1, 10, i+1)
    plt.imshow(img, cmap="gray", interpolation="none")
    plt.xlabel(target[i].item(), fontsize=18)
    plt.xticks([])
    plt.yticks([])

Eseguiamo la funzione di addestramento.

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MyLeNet = MyLeNet.to(device)

y_1,y_2,x = train_cnn(MyLeNet, train_loader, test_loader, device, lr=2e-3, num_epochs=15)

Visualizziamo le **performance** ottenute.

In [0]:
def plot(x_axis,x_label,y_1_axis,y_1_label,curve_label_1,y_2_axis,y_2_label,curve_label_2 ):
   plt.plot(x_axis, y_1_axis, label=curve_label_1)
   plt.plot(x_axis, y_2_axis, label=curve_label_2)
   plt.xlabel(x_label)
   plt.ylabel(y_1_label)
   plt.ylabel(y_2_label)
   plt.legend(shadow=True)

plot(x_axis=x,x_label="epoch",
     y_1_axis=y_1,y_1_label="accuracy",curve_label_1="Train accuracy",
     y_2_axis=y_2,y_2_label="accuracy",curve_label_2="Test accuracy"
     )

Infine visualizziamo qualche **predizione** della nostra rete neurale sull'insieme di test.

In [0]:
def visualize_predictions(model, dataloader, device):
    data, labels = next(iter(dataloader)) 
    data, labels = data[:10].to(device), labels[:10]
    predictions = model(data).max(1)[1]
    
    predictions, data = predictions.cpu(), data.cpu()
    
    plt.figure(figsize=(16,9))
    for i in range(10):
        img = data.squeeze(1)[i]
        plt.subplot(1, 10, i+1)
        plt.imshow(img, cmap="gray", interpolation="none")
        plt.xlabel(predictions[i].item(), fontsize=18)
        plt.xticks([])
        plt.yticks([])    
    
visualize_predictions(MyLeNet, test_loader, device)

**Osservazione**: Nella precedente funzione si fa uso degli **iteratori** per scorrere `dataloader`.

**Esercizio**: Ripetere la classificazione usando *VGG16* al posto di *LeNet-5*.

[Very Deep Convolutional Networks for Large-Scale Visual Recognition](https://www.robots.ox.ac.uk/~vgg/research/very_deep/)